In [34]:
# Dependency conflicts
!pip install dask==2021.4.0 sparse==0.9.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 941.1/941.1 kB 6.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: sparse
    Found existing installation: sparse 0.15.1
    Uninstalling sparse-0.15.1:
      Successfully uninstalled sparse-0.15.1
  Attempting uninstall: dask
    Found existing installation: dask 2023.11.0
    Uninstalling dask-2023.11.0:
      Successfully uninstalled dask-2023.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2023.11.0 requires dask==2023.11.0, but you have dask 2021.4.0 which is incompatible.
rapids-dask-dependency 23.12.1 requires dask==2023.11.0, but you have dask 2021.4.0 which is incompatible.


In [2]:
import os
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from scipy.sparse import csr_matrix, issparse, csc_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
import random
import sc_toolbox
import decoupler as dc
import rpy2.robjects as ro
import diffxpy.api as de
import batchglm
import rpy2.rinterface_lib.callbacks
import anndata2ri
import logging

from rpy2.robjects import pandas2ri
from rpy2.robjects import r

from batchglm.pkg_constants import TF_CONFIG_PROTO

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython
os.chdir('/root/host_home')


2025-05-06 10:14:47.674637: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
%%R
library(edgeR)
library(scater)
library(MAST)
library(Seurat)
library(DESeq2)
library(data.table)
library(dplyr)
#library(EnhancedVolcano)
library(apeglm)
library(lme4)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

Loading required package: limma
Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: MatrixGenerics
Loading required package: matrixStats

Attaching package: ‘MatrixGenerics’

The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2

In [4]:
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
#sc.logging.print_versions()



#logging.getLogger("tensorflow").setLevel(logging.ERROR)
#logging.getLogger("batchglm").setLevel(logging.INFO)


pd.set_option('display.max_rows', 500)
#warnings.filterwarnings("ignore", category=DeprecationWarning, module="tensorflow")

In [5]:
#Set number of threads
TF_CONFIG_PROTO.inter_op_parallelism_threads = 1
TF_CONFIG_PROTO.intra_op_parallelism_threads = 12

# Filenames

In [6]:
#input
adata_input = 'hp_NPCs/processed_data/hp_NPC_clustered_scvi_cuda_soup.h5ad'

In [7]:
#output
save_dir = 'hp_NPCs/tables/' 
resolution = 'cell_types'

# Read in data

In [8]:
adata = sc.read(adata_input)


In [9]:
adata

AnnData object with n_obs × n_vars = 30557 × 16774
    obs: 'lane', 'Demux_state', 'Donor', 'cell_types', 'condition', 'age', 'years_of_illness', 'deltaHY', 'deltaUPDRS', 'deltaADL', 'deltaDOPA', 'gender', 'cultivation_period', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'batch', 'size_factors', 'S_score', 'G2M_score', 'phase', '_scvi_batch', '_scvi_labels', 'Ferroptosis_pos_corr', 'Ferroptosis_neg_corr', 'Apoptosis_pos_corr', 'Apoptosis_neg_corr', 'Necroptosis_pos_corr', 'Autophagy_pos_corr', 'Pathanatos_pos_corr', 'cell_types_abbrev'
    var: 'gene_ids', 'feature_types', 'genome', 'gene_symbols', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_co

# Preprocess data

In [9]:
# Clean up cell type names
adata.obs['cell_types_used'] = [ct.replace(" ", "-") for ct in adata.obs[resolution]]
adata.obs['cell_types_used'] = [ct.replace("+", "") for ct in adata.obs['cell_types_used']]

In [10]:
adata.obs["Donor"] = adata.obs["Donor"].astype("category")
adata.obs["condition"] = adata.obs["condition"].astype("category")
adata.obs['cell_types_used'] = adata.obs['cell_types_used'].astype("category")

In [11]:
adata.obs['cell_types_used'].cat.categories

Index(['Apop.-NCSC', 'Apop.-NSC', 'Glial-precursors', 'Immature-neurons',
       'NCSC', 'NSC1a', 'NSC1b', 'NSC2a', 'NSC2b'],
      dtype='object')

# Check the data

In [12]:
np.mean(adata.layers['soupX_counts'].astype(int) != adata.layers['soupX_counts'])

0.0

Counts are actually all integer counts.

In [13]:
adata.obs.condition.value_counts()

condition
Parkinson    18887
Control      11670
Name: count, dtype: int64

In [14]:
adata.obs.groupby(['cell_types_used'])['condition'].value_counts()

cell_types_used   condition
Apop.-NCSC        Control       110
                  Parkinson     101
Apop.-NSC         Parkinson     964
                  Control       421
Glial-precursors  Parkinson     838
                  Control       153
Immature-neurons  Parkinson     399
                  Control       105
NCSC              Control      2753
                  Parkinson    2251
NSC1a             Parkinson    4848
                  Control      3845
NSC1b             Parkinson    4339
                  Control      1860
NSC2a             Parkinson    3676
                  Control      1971
NSC2b             Parkinson    1471
                  Control       452
Name: count, dtype: int64

In [15]:
adata.obs.groupby(['cell_types_used'])['Donor'].value_counts()

cell_types_used   Donor          
Apop.-NCSC        UKERi1E4-R1-003      63
                  UKERi88H-R1-002      59
                  UKERi82A-R1-002      46
                  UKERiAY6-R1-003      14
                  UKERiC99-R1-007      12
                  UKERiR66-R1-007       7
                  UKERiJ2C-R1-015       5
                  UKERiPX7-R1-001       4
                  UKERiO3H-R1-005       1
                  UKERi1JF-R1-018       0
                  UKERiG3G-R1-039       0
                  UKERiM89-R1-005       0
Apop.-NSC         UKERiJ2C-R1-015     297
                  UKERiAY6-R1-003     247
                  UKERiG3G-R1-039     223
                  UKERiC99-R1-007     165
                  UKERiR66-R1-007     156
                  UKERiO3H-R1-005      83
                  UKERi82A-R1-002      56
                  UKERi88H-R1-002      52
                  UKERiM89-R1-005      33
                  UKERi1E4-R1-003      31
                  UKERi1JF-R1-018      28


Note:
- General imbalance between Parkinson and control cell numbers


# Single-cell specific using diffxpy

In [15]:
if not os.path.exists(save_dir+'diffxpy'):
    # Create a new directory because it does not exist
    os.mkdir(save_dir+'diffxpy')
    print("The new directory: "+save_dir+"diffxpy is created!")
else: 
    print("The directory already exists")

The directory already exists


## Differential expression testing

In [16]:
de_results = dict()


In [17]:
de.utils.preview_coef_names(
    sample_description=adata.obs,
    formula="~1+condition+Donor", 
)

['Intercept',
 'condition[T.Parkinson]',
 'Donor[T.UKERiG3G-R1-039]',
 'Donor[T.UKERi1E4-R1-003]',
 'Donor[T.UKERiO3H-R1-005]',
 'Donor[T.UKERi82A-R1-002]',
 'Donor[T.UKERiJ2C-R1-015]',
 'Donor[T.UKERiM89-R1-005]',
 'Donor[T.UKERiC99-R1-007]',
 'Donor[T.UKERiR66-R1-007]',
 'Donor[T.UKERiAY6-R1-003]',
 'Donor[T.UKERiPX7-R1-001]',
 'Donor[T.UKERi88H-R1-002]']

In [18]:
for clust in adata.obs['cell_types_used'].cat.categories:
    adata_tmp = adata[adata.obs['cell_types_used'] == clust,:].copy()

    print(f'In cluster {clust}:')
    print(adata_tmp.obs.condition.value_counts())

    # Filter out genes to reduce multiple testing burden
    sc.pp.filter_genes(adata_tmp, min_cells=50)
    print(f'Testing {adata_tmp.n_vars} genes...')
    print("")
    
    test_tmp = de.test.wald(
    data=adata_tmp.layers['soupX_counts'],
    formula_loc="~ 1 + condition + Donor + size_factors",
    as_numeric=['size_factors'],
    coef_to_test=["condition[T.Parkinson]"],
    sample_description=adata_tmp.obs,
    gene_names=adata_tmp.var['gene_ids'].astype(str),
    constraints_loc={'Donor':'condition'},
    noise_model='nb',
    dtype="float64"
    )
    
    #Store the results
    de_results[clust] = test_tmp
    

In cluster Apop.-NCSC:
condition
Control      110
Parkinson    101
Name: count, dtype: int64
filtered out 14807 genes that are detected in less than 50 cells
Testing 1967 genes...



Built constraints: Donor[UKERi1E4-R1-003]+Donor[UKERiO3H-R1-005]+Donor[UKERi82A-R1-002]=0, Donor[UKERiJ2C-R1-015]+Donor[UKERiC99-R1-007]+Donor[UKERiR66-R1-007]+Donor[UKERiAY6-R1-003]+Donor[UKERiPX7-R1-001]+Donor[UKERi88H-R1-002]=0


training location model: True
training scale model: True
iter   0: ll=5719522.764339
iter   1: ll=5696145.373691, converged: 0.00% (loc: 2.24%, scale update: False), in 0.57sec
iter   2: ll=5691539.923546, converged: 0.00% (loc: 12.35%, scale update: False), in 0.62sec
iter   3: ll=5690645.405110, converged: 0.00% (loc: 12.35%, scale update: False), in 0.66sec
iter   4: ll=5690452.938784, converged: 0.00% (loc: 12.35%, scale update: False), in 0.59sec
iter   5: ll=5690407.058401, converged: 0.00% (loc: 12.51%, scale update: False), in 0.56sec
Fitting 1967 dispersion models: (progress not available with multiprocessing)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter   6: ll=502715.059239, converged: 0.15% (loc: 0.15%, scale update: True), in 5.11sec
iter   7: ll=501871.722525, converged: 0.15% (loc: 5.85%, scale update: False), in 0.56sec
iter   8: ll=501658.323852, converged: 0.15% (loc: 8.24%, scale update: False), in 0.67sec
iter   9: ll=501607.255001, converged: 0.15% (loc: 11.34%, scale update: False), in 0.56sec
iter  10: ll=501593.655117, converged: 0.15% (loc: 16.22%, scale update: False), in 0.58sec
iter  11: ll=501590.727736, converged: 0.15% (loc: 22.06%, scale update: False), in 0.57sec
Fitting 1964 dispersion models: (progress not available with multiprocessing)

...

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  33: ll=21062864.033231, converged: 99.90% (loc: 99.90%, scale update: True), in 8.23sec
iter  34: ll=21062864.033231, converged: 99.90% (loc: 100.00%, scale update: False), in 0.68sec
Fitting 13 dispersion models: (progress not available with multiprocessing)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  35: ll=21062864.033231, converged: 100.00% (loc: 100.00%, scale update: True), in 5.53sec


/usr/local/lib/python3.10/dist-packages/dask/array/core.py:2894: RuntimeWarning: divide by zero encountered in divide
  size = (limit / dtype.itemsize / largest_block) ** (1 / len(autos))


### Bulk-like DE testing across all clusters

In [18]:
# Make a copy for the testing
adata_test = adata.copy()

print(adata_test.obs.condition.value_counts())

# Filter out genes to reduce multiple testing burden
sc.pp.filter_genes(adata_test, min_cells=50)
print(f'Testing {adata_test.n_vars} genes...')
print("")
    
test_bulk = de.test.wald(
    data=adata_test.layers['soupX_counts'],
    formula_loc="~ 1 + condition + Donor + size_factors",
    as_numeric=['size_factors'],
    coef_to_test=["condition[T.Parkinson]"],
    sample_description=adata_test.obs,
    gene_names=adata_test.var['gene_ids'].astype(str),
    constraints_loc={'Donor':'condition'},
    noise_model='nb',
    dtype="float64"
)


condition
Parkinson    18887
Control      11670
Name: count, dtype: int64
filtered out 907 genes that are detected in less than 50 cells


Built constraints: Donor[UKERi1JF-R1-018]+Donor[UKERiG3G-R1-039]+Donor[UKERi1E4-R1-003]+Donor[UKERiO3H-R1-005]+Donor[UKERi82A-R1-002]=0, Donor[UKERiJ2C-R1-015]+Donor[UKERiM89-R1-005]+Donor[UKERiC99-R1-007]+Donor[UKERiR66-R1-007]+Donor[UKERiAY6-R1-003]+Donor[UKERiPX7-R1-001]+Donor[UKERi88H-R1-002]=0


Testing 15867 genes...



/usr/local/lib/python3.10/dist-packages/batchglm/models/base_glm/utils.py:158: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  np.mean(x[np.where(grouping == g)[0], :], axis=0)
/usr/local/lib/python3.10/dist-packages/batchglm/models/base_glm/utils.py:171: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  expect_xsq = np.vstack([

training location model: True
training scale model: True
iter   0: ll=363734882.856952
iter   1: ll=348656777.751425, converged: 0.00% (loc: 1.47%, scale update: False), in 208.69sec
iter   2: ll=346977047.669265, converged: 0.00% (loc: 3.91%, scale update: False), in 177.61sec
iter   3: ll=346854662.451878, converged: 0.00% (loc: 3.91%, scale update: False), in 169.58sec
iter   4: ll=346828857.640932, converged: 0.00% (loc: 4.16%, scale update: False), in 168.41sec
iter   5: ll=346820795.117586, converged: 0.00% (loc: 20.64%, scale update: False), in 167.35sec
Fitting 15867 dispersion models: (progress not available with multiprocessing)

...

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  36: ll=322372816.846365, converged: 99.93% (loc: 99.93%, scale update: True), in 115.14sec
iter  37: ll=322372816.844150, converged: 99.93% (loc: 99.98%, scale update: False), in 3.28sec
iter  38: ll=322372816.844113, converged: 99.93% (loc: 99.99%, scale update: False), in 2.51sec
iter  39: ll=322372816.844112, converged: 99.93% (loc: 99.99%, scale update: False), in 2.07sec
iter  40: ll=322372816.844112, converged: 99.93% (loc: 100.00%, scale update: False), in 2.00sec
Fitting 11 dispersion models: (progress not available with multiprocessing)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


iter  41: ll=322372467.424401, converged: 99.98% (loc: 99.98%, scale update: True), in 100.86sec
iter  42: ll=322372467.424401, converged: 99.98% (loc: 100.00%, scale update: False), in 2.42sec
iter  43: ll=322372467.424401, converged: 100.00% (loc: 100.00%, scale update: True), in 4.06sec


/usr/local/lib/python3.10/dist-packages/dask/array/core.py:2894: RuntimeWarning: divide by zero encountered in divide
  size = (limit / dtype.itemsize / largest_block) ** (1 / len(autos))


## Check some data

In [20]:
de_results['NCSC'].summary().sort_values(by='qval', ascending=True).iloc[:10,:]

,gene,pval,qval,log2fc,mean,zero_mean,grad,coef_mle,coef_sd,ll
5178,ENSG00000004948,0.0,0.0,2.950221,0.031375,False,4.075215e-03,2.950221,2.222759e-162,-282.914000
3609,ENSG00000145526,0.0,0.0,3.144584,0.022382,False,6.430340e-06,3.144584,2.222759e-162,-354.186377
12309,ENSG00000069399,0.0,0.0,4.822501,0.027978,False,2.366384e-04,4.822501,2.222759e-162,-602.643831
11206,ENSG00000108684,0.0,0.0,13.174367,0.015388,False,8.540608e-06,13.174367,2.222759e-162,-309.792544
7366,ENSG00000080854,0.0,0.0,0.904927,0.031775,False,1.650007e-04,0.904927,2.222759e-162,-587.151130
11998,ENSG00000101265,0.0,0.0,10.733867,0.035771,False,9.558911e-08,10.733867,2.222759e-162,-728.736454
12773,ENSG00000267179,0.0,0.0,7.579408,0.020584,False,1.738267e-04,7.579408,2.222759e-162,-496.422021
2958,ENSG00000080224,0.0,0.0,-3.229672,0.016187,False,2.828667e-05,-3.229672,2.222759e-162,-373.013897
6453,ENSG00000259305,0.0,0.0,11.248681,0.033373,False,7.492728e-07,11.248681,2.222759e-162,-716.945059
7360,ENSG00000110090,0.0,0.0,2.889355,0.026779,False,1.059688e-03,2.889355,2.222759e-162,-575.324484


In [21]:
# Test if model did not converger for some genes and some clusters
np.sum(de_results['NCSC'].summary()['ll'] == np.float64('-inf'))
np.sum(de_results['NSC2a'].summary()['ll'] == np.float64('-inf'))

0

0

In [19]:
test_bulk.summary().sort_values(by='qval', ascending=True).iloc[:10,:]

,gene,pval,qval,log2fc,mean,zero_mean,grad,coef_mle,coef_sd,ll
7933,ENSG00000165029,0.0,0.0,0.232584,0.610466,False,9.786149e-08,0.232584,0.022677,-30649.902072
8170,ENSG00000120162,0.0,0.0,0.227735,0.835030,False,3.432685e-07,0.227735,0.023194,-36213.117918
12317,ENSG00000066654,0.0,0.0,0.162000,0.576136,False,2.796505e-07,0.162000,0.019270,-29636.510819
3872,ENSG00000138823,0.0,0.0,0.383701,0.218804,False,2.739176e-08,0.383701,0.037369,-16483.217086
12319,ENSG00000183044,0.0,0.0,0.325313,0.315247,False,1.741691e-07,0.325313,0.028673,-21370.311338
12321,ENSG00000153048,0.0,0.0,0.235369,2.495337,False,1.926697e-06,0.235369,0.010784,-57357.036490
12322,ENSG00000086504,0.0,0.0,0.123085,1.318618,False,4.288843e-07,0.123085,0.012667,-44208.300747
3873,ENSG00000170365,0.0,0.0,-0.157020,0.596099,False,2.661180e-07,-0.157020,0.018675,-30535.683681
1640,ENSG00000115350,0.0,0.0,0.122205,1.190136,False,6.976995e-07,0.122205,0.014225,-42659.814567
12324,ENSG00000103202,0.0,0.0,-0.136223,4.993978,False,6.417595e-07,-0.136223,0.008181,-73142.340749


## Post-process data

In [20]:
# extract gene symbol - gene_ids informations for later conversion
df_var = pd.DataFrame(adata.var)
df_var = df_var[['gene_symbols', 'gene_ids']]
df_var.reset_index(inplace=True) 
df_var = df_var.drop(columns='index')

In [24]:
de_signif = dict()
de_full_res = dict()

for clust in de_results:
    tmp = de_results[clust].summary()
    
    # add gene symbols
    tmp = tmp.rename(columns = {"gene" : "gene_ids"})
    tmp = tmp.merge(df_var, how = 'inner', on='gene_ids')

    n_infll = np.sum(tmp['ll'] == np.float64('-inf'))
    print(f'There are {n_infll} -inf log likelihood values. Filtering these out now...')
    tmp = tmp.loc[tmp['ll'] != np.float64('-inf')]

    n_unconverged = np.sum(tmp['coef_sd'] < 10E-100)
    print(f'There are {n_unconverged} logFC std. dev.s < 10E-100. Filtering these out now...')    
    tmp = tmp.loc[tmp['coef_sd'] > 10E-100]
    
    #Sort values by q-value
    tmp.sort_values(by='qval', ascending=True, inplace=True)
    
    de_full_res[clust] = tmp.copy()
    
    #Filter by q-value to get significant DE genes
    tmp_filt = tmp.loc[tmp['qval'] < 0.05]
    de_signif[clust] = tmp_filt.copy()

There are 0 -inf log likelihood values. Filtering these out now...
There are 61 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 1 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 5 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 0 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 39 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 1 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out now...
There are 2 logFC std. dev.s < 10E-100. Filtering these out now...
There are 0 -inf log likelihood values. Filtering these out 

In [21]:
# bulk-like data
bulk_res = test_bulk.summary()

# add gene symbols
bulk_res = bulk_res.rename(columns = {"gene" : "gene_ids"})
bulk_res = bulk_res.merge(df_var, how = 'inner', on='gene_ids')

n_infll = np.sum(bulk_res['ll'] == np.float64('-inf'))
print(f'There are {n_infll} -inf log likelihood values. Filtering these out now...')
bulk_res = bulk_res.loc[bulk_res['ll'] != np.float64('-inf')]

n_unconverged = np.sum(bulk_res['coef_sd'] < 10E-100)
print(f'There are {n_unconverged} logFC std. dev.s < 10E-100. Filtering these out now...')    
bulk_res = bulk_res.loc[bulk_res['coef_sd'] > 10E-100]

#Sort values by q-value
bulk_res.sort_values(by='qval', ascending=True, inplace=True)
        
#Filter by q-value to get significant DE genes
bulk_res_filt = bulk_res.loc[bulk_res['qval'] < 0.05]


There are 0 -inf log likelihood values. Filtering these out now...
There are 0 logFC std. dev.s < 10E-100. Filtering these out now...


## Show DEGs

In [26]:
# Number of differentially expressed genes (DEGs)
print('DEGs at FDR < 0.05:')
{clust:de_signif[clust].shape[0] for clust in de_signif}

print('DEGs at FDR < 0.01:')
{clust:de_signif[clust].loc[(de_signif[clust]['qval']<0.01)].shape[0] for clust in de_signif}

DEGs at FDR < 0.05:


{'Apop.-NCSC': 5,
 'Apop.-NSC': 88,
 'Glial-precursors': 3,
 'Immature-neurons': 24,
 'NCSC': 8,
 'NSC1a': 2373,
 'NSC1b': 3081,
 'NSC2a': 3745,
 'NSC2b': 1568}

DEGs at FDR < 0.01:


{'Apop.-NCSC': 0,
 'Apop.-NSC': 49,
 'Glial-precursors': 1,
 'Immature-neurons': 14,
 'NCSC': 5,
 'NSC1a': 1633,
 'NSC1b': 2007,
 'NSC2a': 2638,
 'NSC2b': 947}

In [22]:
#DEGs in bulk-like data
print('DEGs at FDR < 0.05:')
print(f'Bulk: {bulk_res_filt.shape[0]}')

print('DEGs at FDR < 0.01:')
degnum001 = bulk_res_filt.loc[(bulk_res_filt['qval']<0.01)].shape[0]
print(f'Bulk: {degnum001}')

DEGs at FDR < 0.05:
Bulk: 8281
DEGs at FDR < 0.01:
Bulk: 6931


Notes:
- there are quite a lot of DEGs at FDR < 0.05 and at FDR < 0.01
- Could use either threshold, FDR < 0.01 would be okay for higher confidence

## Save data

In [28]:
import pickle

In [29]:
#Pickle the de output
filename_pickle = save_dir+'diffxpy/'+resolution+'_diffxpy_cell_types_sf.pkl'
pickle.dump(de_results, open(filename_pickle, "wb"))

In [30]:
#Output individual results
for clust in de_signif:
    
    filename_signif = save_dir+'diffxpy/'+resolution+'_diffxpy_'+clust.replace(' ', '_').replace(
        '/','_').replace('(','').replace(')','')+'_diff_res.csv'
    filename_full = save_dir+'diffxpy/'+resolution+'_diffxpy_'+clust.replace(' ', '_').replace(
        '/','_').replace('(','').replace(')','')+'_res-table.csv'
    
    de_signif[clust].to_csv(filename_signif)
    de_full_res[clust].to_csv(filename_full)

In [31]:
#Pick the DE output - Bulk-like
de_output_pickle_bulk = save_dir+'diffxpy/'+resolution+'_diffxpy_bulk_sf.pkl'
pickle.dump(test_bulk, open(de_output_pickle_bulk, "wb"))

In [23]:
#Output individual results - Bulk-like

filename_signif = save_dir+'diffxpy/'+resolution+'_diffxpy_bulk_like_diff_res.csv'
filename_full = save_dir+'diffxpy/'+resolution+'_diffxpy_bulk_like_res-table.csv'
    
bulk_res_filt.to_csv(filename_signif)
bulk_res.to_csv(filename_full)

In [33]:
!pip install dask==2023.11.0 sparse==0.15.1

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: sparse
    Found existing installation: sparse 0.9.1
    Uninstalling sparse-0.9.1:
      Successfully uninstalled sparse-0.9.1
  Attempting uninstall: dask
    Found existing installation: dask 2021.4.0
    Uninstalling dask-2021.4.0:
      Successfully uninstalled dask-2021.4.0
